In [1]:
import pandas as pd
import numpy as np

In [15]:
class DataExtractor():
    def __init__(self, path) -> None:
        self.path = path
        self.final = pd.DataFrame()
        pass
    
    def extractDataFrame(self):
        
        def NotaNum(quant):
            if(not str(quant).isnumeric()):
                if(str(quant) == "ten"):
                    return 10
                if(str(quant) == "five"):
                    return 5
                return 0
            return quant
    
        def removeLastletter(id):
            if(not str(id)[-1].isnumeric()):
                return id[:-1]
            return id
    
        df = pd.read_pickle(self.path)
        df = pd.DataFrame(df)
        
        df.dropna(inplace=True)
        df.reset_index(inplace=True)
        
        df["id"] = df["id"].map(removeLastletter)

        final = pd.DataFrame()
        items = pd.DataFrame()
        for i in range(0, df.shape[0]):
            for k in range(0, len(df["items"][i])):
                item = pd.DataFrame(df["items"][i][k]["item"], index = [df["id"][i]])
                item["quantity"] = df["items"][i][k]["quantity"]
                item["created_on"] = df["created_on"][i]
                items = pd.concat([items, item])
            final = pd.concat([final, items])
            items = pd.DataFrame()

        final.index.name = "invoice_id"
        final.rename(columns={"id":"invoiceitem_id", "name":"invoiceitem_name"}, inplace=True)
        final.index = final.index.astype(int)

        final["invoiceitem_name"] = final["invoiceitem_name"].astype(str)

        final["created_on"] = pd.to_datetime(final["created_on"], format="mixed", errors='coerce')

        final["quantity"] = final["quantity"].map(NotaNum).astype(int)

        final['type'] = final['type'].map({0: 'Material', 1: 'Equipment', 2: 'Service', 3: 'Other'})

        final["total_price"] = final["unit_price"] * final["quantity"]

        final.sort_values(["invoice_id", "invoiceitem_id"], inplace=True)

        final["percentage_in_invoice"] = final["total_price"] / final.groupby('invoice_id')['total_price'].transform('sum')

        lines = np.loadtxt("expired_invoices.txt", comments="#", delimiter=",", unpack=False, dtype=np.int64)
        final["is_expired"] = final.index.map(lambda x: x in lines)
        
        final = final[["created_on", "invoiceitem_id", "invoiceitem_name", 
                       "type", "unit_price", "total_price", "percentage_in_invoice", "is_expired"]]
        self.final = final
        return final

    def save_csv(self):
        if not self.final.empty:
            self.final.to_csv("out.csv")
        else:
            print("Extract the Dataset first")
    
        
        

In [16]:
data = DataExtractor("invoices_new.pkl")
data.extractDataFrame()
data.save_csv()